In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import optuna

In [3]:
import os
import random
import numpy as np
import torch

# Set a fixed seed value
seed_value = 10

# 1. Set PYTHONHASHSEED environment variable for reproducibility in hashing
os.environ['PYTHONHASHSEED'] = str(seed_value)

# 2. Set the seed for Python's built-in random module
random.seed(10)

# 3. Set the seed for NumPy
np.random.seed(10)

# 4. Set the seed for PyTorch
torch.manual_seed(seed_value)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)

# 5. Force PyTorch to use deterministic algorithms (may impact performance)
torch.use_deterministic_algorithms(True, warn_only=True)

# 6. Optionally limit the number of threads used by OMP and MKL (helps reduce non-determinism)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

print("Seeds and environment variables set for reproducibility.")

Seeds and environment variables set for reproducibility.


In [5]:
nba_series = nba_series = pd.read_csv("TEAM_MATCHUP_DATA_CLUSTER.csv") # updated data set with CLUSTERING PROPORTIONS (CHECK BOX)

nba_series['SEASON_YEAR'] = nba_series['SEASON'].str.split('-').str[0].astype(int)
nba_series['SEASON'] = (nba_series['SEASON_YEAR'] + 1).astype(int)
nba_series = nba_series.drop(columns=['SEASON_YEAR'])

In [7]:
# Drop columns that are not needed
nba_series = nba_series.drop(columns = ['SERIES_ID', 'SEASON_ID', 'TEAM_1_ID', 'TEAM_2_ID', 'CLUSTER_TEAM_1', 'CLUSTER_TEAM_2'])

# Create a mask to flip half of the rows
flip_mask = np.random.rand(len(nba_series)) < 0.5

# Columns to swap
team1_stat_cols = [col for col in nba_series.columns if '_TEAM_1' in col]
team2_stat_cols = [col.replace('_TEAM_1', '_TEAM_2') for col in team1_stat_cols]

# Include team name columns for flipping
stat_swap_cols = team1_stat_cols + team2_stat_cols + ['TEAM_1', 'TEAM_2']

# Create deep copies of swapped and non-swapped rows
swapped = nba_series.loc[flip_mask].copy()
not_swapped = nba_series.loc[~flip_mask].copy()

# Flip stats
swapped[team1_stat_cols] = nba_series.loc[flip_mask, team2_stat_cols].values
swapped[team2_stat_cols] = nba_series.loc[flip_mask, team1_stat_cols].values

# Flip team names
swapped['TEAM_1'] = nba_series.loc[flip_mask, 'TEAM_2'].values
swapped['TEAM_2'] = nba_series.loc[flip_mask, 'TEAM_1'].values

# Recalculate TEAM_1_W based on new TEAM_1 vs SERIES_WINNER
swapped['TEAM_1_W'] = (swapped['SERIES_WINNER'] == swapped['TEAM_1']).astype(int)
not_swapped['TEAM_1_W'] = (not_swapped['SERIES_WINNER'] == not_swapped['TEAM_1']).astype(int)

# Combine flipped and unflipped
nba_series_balanced = pd.concat([swapped, not_swapped], ignore_index=True)

# Optional: Shuffle the final DataFrame
nba_series_balanced = nba_series_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# test_data_preserved is created for displaying results later on
test_data_preserved = nba_series_balanced[['SEASON', 'TEAM_1', 'TEAM_2', 'SERIES_WINNER']]
test_data_preserved = test_data_preserved

C:\Users\Sarvesh\AppData\Local\Temp\ipykernel_20820\651144542.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  swapped['TEAM_1_W'] = (swapped['SERIES_WINNER'] == swapped['TEAM_1']).astype(int)


In [9]:
# Automatically extract stat bases by checking for matching suffixes
diff_df = pd.DataFrame()

# Grab all columns ending in _TEAM_1
team1_cols = [col for col in nba_series_balanced.columns if col.endswith('_TEAM_1')]

for col1 in team1_cols:
    # Get the base stat name (e.g., 'AST', 'FG_PCT')
    stat_base = col1.replace('_TEAM_1', '')
    col2 = f'{stat_base}_TEAM_2'
    
    # Only compute diff if TEAM_2 version exists
    if col2 in nba_series_balanced.columns:
        diff_df[f'{stat_base}_DIFF'] = nba_series_balanced[col1] - nba_series_balanced[col2]

# Add label and season columns
diff_df['TEAM_1_W'] = nba_series_balanced['TEAM_1_W']
diff_df['SEASON'] = nba_series_balanced['SEASON']

In [23]:
import optuna
import numpy as np
import pandas as pd
import os
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from pytorch_tabnet.tab_model import TabNetClassifier
import warnings
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Ensure save directory exists
os.makedirs("saved_models", exist_ok=True)

years = np.arange(2020, 2025)

def objective_tabnet(trial):
    
    n_d = trial.suggest_int("n_d", 8, 64, step=8)
    n_a = trial.suggest_int("n_a", 8, 64, step=8)
    n_steps = trial.suggest_int("n_steps", 3, 10)
    gamma = trial.suggest_float("gamma", 1.0, 2.0)
    lambda_sparse = trial.suggest_float("lambda_sparse", 0.00001, 0.01, log=True)
    momentum = trial.suggest_float("momentum", 0.01, 0.4)
    lr = trial.suggest_float("lr", 1e-3, 2e-2, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    mask_type = trial.suggest_categorical("mask_type", ['sparsemax', 'entmax'])
    

    '''
    n_d = trial.suggest_int("n_d", 40, 56, step=4)
    n_a = trial.suggest_int("n_a", 40, 56, step=4)
    n_steps = trial.suggest_int("n_steps", 6, 10)
    gamma = trial.suggest_float("gamma", 1.0, 1.2)
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-5, 1e-3, log=True)
    momentum = trial.suggest_float("momentum", 0.3, 0.4)
    lr = trial.suggest_float("lr", 0.001, 0.003, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-5, log=True)
    mask_type = trial.suggest_categorical("mask_type", ['sparsemax', 'entmax'])
    '''

    # Store predictions and true labels for global metrics
    all_y_true = []
    all_y_pred = []
    all_y_prob = []

    for year in years:
        train_data = diff_df[diff_df['SEASON'] < year]
        test_data = diff_df[diff_df['SEASON'] == year]

        if train_data.empty or test_data.empty:
            continue

        X_train = train_data.drop(columns=['TEAM_1_W', 'SEASON'])
        y_train = train_data['TEAM_1_W']
        X_test = test_data.drop(columns=['TEAM_1_W', 'SEASON'])
        y_test = test_data['TEAM_1_W']

        

        smote = SMOTE(random_state=10)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

        clf = TabNetClassifier(
            n_d=n_d, n_a=n_a, n_steps=n_steps,
            gamma=gamma, lambda_sparse=lambda_sparse,
            momentum=momentum, mask_type=mask_type,
            optimizer_params={"lr": lr, "weight_decay": weight_decay},
            seed=10, verbose=0, device_name='cpu'
        )

        clf.fit(
            X_train_resampled.values, y_train_resampled.values,
            eval_set=[(X_test.values, y_test.values)],
            eval_metric=['auc'],
            max_epochs=200, patience=20,
            batch_size=64, virtual_batch_size=32
        )

        X_test_flip = -X_test.values
        y_probs_flip = clf.predict_proba(X_test_flip)[:, 1]

        # Symmetric inference: average regular and 1 - flipped
        symmetric_probs = (clf.predict_proba(X_test.values)[:, 1] + (1 - y_probs_flip)) / 2
        y_pred = (symmetric_probs >= 0.5).astype(int)

        #y_probs = clf.predict_proba(X_test.values)[:, 1]
        #y_pred = (y_probs >= 0.5).astype(int)

        #preds_label = clf.predict(X_test.values)

        # Collect for global metrics
        all_y_true.extend(y_test.values)
        all_y_pred.extend(y_pred)
        all_y_prob.extend(symmetric_probs)

    if len(all_y_true) == 0:
        # Skip trial if no data
        return 0.0

    # Compute global metrics
    global_auc = roc_auc_score(all_y_true, all_y_prob)
    global_accuracy = accuracy_score(all_y_true, all_y_pred)
    global_f1 = f1_score(all_y_true, all_y_pred)

    print(f"Trial {trial.number}: Global AUC = {global_auc:.4f}, Accuracy = {global_accuracy:.4f}, F1 = {global_f1:.4f}")

    
    # ✅ Save models with high AUC
    if global_auc >= 0.90:
        filename = f"tabnet_trial{trial.number}_auc{global_auc:.4f}.zip"
        filepath = os.path.join("saved_models", filename)
        clf.save_model(filepath)
        print(f"✅ Saved model for Trial {trial.number} → {filename}")
    

    return global_auc

# Run the Optuna study
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=8))
study.optimize(objective_tabnet, n_trials=200)

# Print best results
print("Best AUC:", study.best_value)
print("Best Params:", study.best_params)

[I 2025-04-24 12:15:36,227] A new study created in memory with name: no-name-40f6214c-6325-41d6-90fb-5fadeabd72c2



Early stopping occurred at epoch 24 with best_epoch = 4 and best_val_0_auc = 0.98214

Early stopping occurred at epoch 22 with best_epoch = 2 and best_val_0_auc = 0.75926

Early stopping occurred at epoch 42 with best_epoch = 22 and best_val_0_auc = 0.81481

Early stopping occurred at epoch 24 with best_epoch = 4 and best_val_0_auc = 0.73214

Early stopping occurred at epoch 39 with best_epoch = 19 and best_val_0_auc = 0.78


[I 2025-04-24 12:17:28,984] Trial 0 finished with value: 0.6714285714285714 and parameters: {'n_d': 56, 'n_a': 64, 'n_steps': 9, 'gamma': 1.5308556915555989, 'lambda_sparse': 4.990970282009197e-05, 'momentum': 0.01444553366819766, 'lr': 0.0036312223666535117, 'weight_decay': 4.068229457500928e-05, 'mask_type': 'sparsemax'}. Best is trial 0 with value: 0.6714285714285714.


Trial 0: Global AUC = 0.6714, Accuracy = 0.6267, F1 = 0.6500

Early stopping occurred at epoch 27 with best_epoch = 7 and best_val_0_auc = 0.73214

Early stopping occurred at epoch 32 with best_epoch = 12 and best_val_0_auc = 0.77778

Early stopping occurred at epoch 44 with best_epoch = 24 and best_val_0_auc = 0.96296

Early stopping occurred at epoch 44 with best_epoch = 24 and best_val_0_auc = 0.75

Early stopping occurred at epoch 29 with best_epoch = 9 and best_val_0_auc = 0.82


[I 2025-04-24 12:19:10,061] Trial 1 finished with value: 0.687857142857143 and parameters: {'n_d': 40, 'n_a': 40, 'n_steps': 9, 'gamma': 1.7123745740820913, 'lambda_sparse': 0.0007228468425177478, 'momentum': 0.17617579048571497, 'lr': 0.002377361812957646, 'weight_decay': 0.007859971382309462, 'mask_type': 'sparsemax'}. Best is trial 1 with value: 0.687857142857143.


...


Early stopping occurred at epoch 31 with best_epoch = 11 and best_val_0_auc = 0.8
Trial 35: Global AUC = 0.7250, Accuracy = 0.6000, F1 = 0.6154

Early stopping occurred at epoch 45 with best_epoch = 25 and best_val_0_auc = 0.83929

Early stopping occurred at epoch 31 with best_epoch = 11 and best_val_0_auc = 0.90741

Early stopping occurred at epoch 33 with best_epoch = 13 and best_val_0_auc = 0.85185

Early stopping occurred at epoch 25 with best_epoch = 5 and best_val_0_auc = 0.60714

Early stopping occurred at epoch 47 with best_epoch = 27 and best_val_0_auc = 0.86


[I 2025-04-24 13:00:54,654] Trial 36 finished with value: 0.7664285714285715 and parameters: {'n_d': 56, 'n_a': 64, 'n_steps': 7, 'gamma': 1.3527722576058043, 'lambda_sparse': 0.003699467963412427, 'momentum': 0.11440995233732523, 'lr': 0.017866701797471258, 'weight_decay': 0.009767681346879763, 'mask_type': 'sparsemax'}. Best is trial 26 with value: 0.8242857142857142.


Trial 36: Global AUC = 0.7664, Accuracy = 0.7067, F1 = 0.7250

Early stopping occurred at epoch 41 with best_epoch = 21 and best_val_0_auc = 0.80357

Early stopping occurred at epoch 23 with best_epoch = 3 and best_val_0_auc = 0.94444

Early stopping occurred at epoch 86 with best_epoch = 66 and best_val_0_auc = 0.88889


[W 2025-04-24 13:01:59,931] Trial 37 failed with parameters: {'n_d': 64, 'n_a': 56, 'n_steps': 5, 'gamma': 1.683919659275522, 'lambda_sparse': 0.002380518927422153, 'momentum': 0.15992899813063532, 'lr': 0.004243765037963861, 'weight_decay': 6.668774492519852e-05, 'mask_type': 'entmax'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Sarvesh\anaconda3\envs\NBACluster\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Sarvesh\AppData\Local\Temp\ipykernel_20820\2116535040.py", line 72, in objective_tabnet
    clf.fit(
  File "C:\Users\Sarvesh\anaconda3\envs\NBACluster\Lib\site-packages\pytorch_tabnet\abstract_model.py", line 262, in fit
    self._predict_epoch(eval_name, valid_dataloader)
  File "C:\Users\Sarvesh\anaconda3\envs\NBACluster\Lib\site-packages\pytorch_tabnet\abstract_model.py", line 562, in _predict_epoch
    scores 

KeyboardInterrupt: 

In [27]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, precision_score, recall_score, log_loss
import numpy as np

# Tracking
all_y_true = []
all_y_prob_blended = []
all_y_pred_blended = []

# Per-year metrics
yearly_metrics = []

years = np.arange(2020, 2025)

# 84% accuracy set (2020-2025 test), 0.883 AUC, threshold = 0.5, seed = 8 (fit, optuna)
params_a = {
    'n_d': 56,
    'n_a': 64,
    'n_steps': 6,
    'gamma': 1.3930814539055605,
    'lambda_sparse': 0.0029833605183872134,
    'momentum': 0.15891701764952307,
    'mask_type': 'entmax'
}

opt_params_a = {
    'lr': 0.0076661428463103455,
    'weight_decay': 2.501056095511525e-05
}

# 85% accuracy set (2020-2025 test), 0.89 AUC, threshold = 0.51, seed = 10 (fit, optuna)
params_b = {
    'n_d': 48,
    'n_a': 48,
    'n_steps': 8,
    'gamma': 1.0950455922432034,
    'lambda_sparse': 0.000365814411562923,
    'momentum': 0.3675391603570311,
    'mask_type': 'sparsemax'
}

opt_params_b = {
    'lr': 0.002071649487926403,
    'weight_decay': 4.2583833531711615e-06
}

# 79% accuracy set (2005-2005 test), 0.869 AUC, threshold = 0.5, seed = 5 (fit, optuna) 
params_c = {
    'n_d': 48,
    'n_a': 32,
    'n_steps': 5,
    'gamma': 1.1885687695822824,
    'lambda_sparse': 0.0019929526182699384,
    'momentum': 0.25773242436863386,
    'mask_type': 'entmax'
}

opt_params_c = {
    'lr': 0.01601995480535204,
    'weight_decay': 0.00015417059849810502,
}


for year in years:
    print(f"\n📆 Evaluating year: {year}")
    train_data = diff_df[diff_df['SEASON'] < year]
    test_data = diff_df[diff_df['SEASON'] == year]
    
    if train_data.empty or test_data.empty:
        continue

    X_train = train_data.drop(columns=['TEAM_1_W', 'SEASON'])
    y_train = train_data['TEAM_1_W']
    X_test = test_data.drop(columns=['TEAM_1_W', 'SEASON'])
    y_test = test_data['TEAM_1_W']

    smote = SMOTE(random_state=10)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # -------- Model A --------
    model_a = TabNetClassifier(
        **params_a,
        optimizer_params=opt_params_a,
        seed=8,
        device_name='cpu',
        verbose=0
    )
    model_a.fit(
        X_train_resampled.values, y_train_resampled.values,
        eval_set=[(X_test.values, y_test.values)],
        eval_metric=['auc'],
        max_epochs=200, patience=20,
        batch_size=64, virtual_batch_size=32
    )
    probs_a = model_a.predict_proba(X_test.values)[:, 1]
    
    
    # -------- Model B --------
    model_b = TabNetClassifier(
        **params_b,
        optimizer_params=opt_params_b,
        seed=10,
        device_name='cpu',
        verbose=0
    )
    model_b.fit(
        X_train_resampled.values, y_train_resampled.values,
        eval_set=[(X_test.values, y_test.values)],
        eval_metric=['auc'],
        max_epochs=200, patience=20,
        batch_size=64, virtual_batch_size=32
    )
    probs_b = model_b.predict_proba(X_test.values)[:, 1]
    
    '''
    # -------- Model C --------
    model_c = TabNetClassifier(
        **params_c,
        optimizer_params=opt_params_c,
        seed=5,
        device_name='cpu',
        verbose=0
    )
    model_c.fit(
        X_train_resampled.values, y_train_resampled.values,
        eval_set=[(X_test.values, y_test.values)],
        eval_metric=['auc'],
        max_epochs=200, patience=20,
        batch_size=64, virtual_batch_size=32
    )
    probs_c = model_c.predict_proba(X_test.values)[:, 1]
    '''
    
    X_test_flipped = -X_test.values

    '''
    # -------- Blend Predictions --------
    blended_probs = (probs_a + probs_b) / 2
    blended_preds = (blended_probs >= 0.505).astype(int)
    '''

    # Flip all the features by negating them
    X_test_flipped = -X_test.values

    # Get predictions for flipped inputs
    probs_a_flip = model_a.predict_proba(X_test_flipped)[:, 1]
    probs_b_flip = model_b.predict_proba(X_test_flipped)[:, 1]
    probs_c_flip = model_a.predict_proba(X_test_flipped)[:, 1]
    
    # Blend forward and flipped predictions separately
    blended_forward = (probs_a + probs_b) / 2
    blended_flipped = 1 - ((probs_a_flip + probs_b_flip) / 2)  # invert because flipped input reverses label

    # Final symmetric probability: average of regular and flipped
    symmetric_probs = (blended_forward + blended_flipped) / 2

    # Convert to binary predictions using threshold
    blended_preds = (symmetric_probs >= 0.52).astype(int)
    blended_probs = symmetric_probs

    bce_loss = log_loss(y_test, blended_probs)

    # Metrics for this year
    auc = roc_auc_score(y_test, blended_probs)
    acc = accuracy_score(y_test, blended_preds)
    f1 = f1_score(y_test, blended_preds)
    cm = confusion_matrix(y_test, blended_preds)

    print(f"AUC: {auc:.4f} | Accuracy: {acc:.4f} | F1: {f1:.4f}")
    print(f"Binary Cross-Entropy Loss: {bce_loss:.4f}")
    print("Confusion Matrix:")
    print(cm)

    yearly_metrics.append({
        "year": year,
        "auc": auc,
        "accuracy": acc,
        "f1": f1,
        "confusion_matrix": cm
    })

    # Global tracking
    all_y_true.extend(y_test)
    all_y_prob_blended.extend(blended_probs)
    all_y_pred_blended.extend(blended_preds)

# ------ Overall Metrics ------
overall_auc = roc_auc_score(all_y_true, all_y_prob_blended)
overall_acc = accuracy_score(all_y_true, all_y_pred_blended)
overall_f1 = f1_score(all_y_true, all_y_pred_blended)
overall_prec = precision_score(all_y_true, all_y_pred_blended)
overall_rec = recall_score(all_y_true, all_y_pred_blended)
overall_bce_loss = log_loss(all_y_true, all_y_prob_blended)

print(f"\n🔁 Overall Blended Model Performance:")
print(f"AUC: {overall_auc:.4f}, Accuracy: {overall_acc:.4f}, F1: {overall_f1:.4f}, Precision: {overall_prec:.4f}, Recall: {overall_rec:.4f}")
print(f"Overall Binary Cross-Entropy Loss: {overall_bce_loss:.4f}")


📆 Evaluating year: 2020

Early stopping occurred at epoch 46 with best_epoch = 26 and best_val_0_auc = 0.875

Early stopping occurred at epoch 48 with best_epoch = 28 and best_val_0_auc = 0.92857
AUC: 0.8393 | Accuracy: 0.8000 | F1: 0.8235
Binary Cross-Entropy Loss: 0.5730
Confusion Matrix:
[[5 2]
 [1 7]]

📆 Evaluating year: 2021

Early stopping occurred at epoch 30 with best_epoch = 10 and best_val_0_auc = 0.88889

Early stopping occurred at epoch 41 with best_epoch = 21 and best_val_0_auc = 0.7963
AUC: 0.8889 | Accuracy: 0.8667 | F1: 0.8571
Binary Cross-Entropy Loss: 0.5945
Confusion Matrix:
[[7 2]
 [0 6]]

📆 Evaluating year: 2022

Early stopping occurred at epoch 29 with best_epoch = 9 and best_val_0_auc = 0.92593

Early stopping occurred at epoch 48 with best_epoch = 28 and best_val_0_auc = 0.96296
AUC: 1.0000 | Accuracy: 0.8667 | F1: 0.8750
Binary Cross-Entropy Loss: 0.4970
Confusion Matrix:
[[6 0]
 [2 7]]

📆 Evaluating year: 2023

Early stopping occurred at epoch 28 with best_ep